# Data fitting

Curve fitting is the process of constructing a curve, or mathematical function, that has the best fit to a series of eperimental data points.

In this exercise we will calculate the sum of the "medipix.edf" image along the vertical and horizontal axis and fit a gaussian function, using ``silx.math``.


## Loading data

In [ ]:
%pylab inline

In [ ]:
from silx.io import spech5
specfile = spech5.SpecH5("31oct98.dat")
xdata = specfile["/22.1/measurement/TZ3"]
ydata = specfile["/22.1/measurement/If4"]
plot(xdata, ydata)

## Use of a builtin fitting function

The fitting function is a function of the data-point (xdata) and of a sert of parameters. Silx offers a set of usual fitting functions in module ``silx.math.fit.functions``.

In this example, we will use a gaussian function, ``sum_gauss``, whose parameters are *height*, *center* an *fwhm* (full-width at half maximum). The functions accepts a multiple number of these 3 base parameters, to fit a sum of multiple gaussian peaks.


In [ ]:
from silx.math.fit.functions import sum_gauss

An essential part of the iterative fitting process is the choice of the initial set of parameters p0. Our initial estimate is that we have 2 peaks, centered at $x=29.45$ and at $x=29.55$, with amplitude $55000$ and $60000$ and with a full-width at half maximum of $0.03$.

In [ ]:
p0 = [55000., 29.45, 0.03,
      60000., 29.55, 0.03]
plot(xdata, ydata, label="Actual data")
plot(xdata, sum_gauss(xdata, *p0), label="Initial guess")
legend()
title("Before the fit:")


## Fitting using silx


In [ ]:
from silx.math.fit import leastsq

In [ ]:
p, cov_matrix = leastsq(model=sum_gauss, xdata=xdata, ydata=ydata, p0=p0)
print(p)

In [ ]:
%timeit p, cov_matrix = leastsq(model=sum_gauss, xdata=xdata, ydata=ydata, p0=p0)

In [ ]:
#Display the result of the fit
#plot(xdata, sum_gauss(xdata, *p0), label="Initial guess")
plot(xdata, sum_gauss(xdata, *p), label="Fitted curve")
plot(xdata, ydata,label="Experimental data")
legend()
title("Result of the fit")

We could probably improve this with a small third peak at $x=29.61$

In [ ]:
# estimate
p0 = [55000., 29.45, 0.03,
      60000., 29.55, 0.03,
      5000., 29.61, 0.05]
# fit
p, cov_matrix = leastsq(model=sum_gauss, xdata=xdata, ydata=ydata, p0=p0)

# plot
plot(xdata, sum_gauss(xdata, *p0), label="Initial guess")
plot(xdata, sum_gauss(xdata, *p), label="Fitted curve")
plot(xdata, ydata, label="Experimental data")
legend()
title("Result of the fit (3 peaks)")


It turns out that we were wrong regarding the position of our third peak, and about the amplitudes of all 3 peaks. The fit converged towards a much better 3-peak solution:

In [ ]:
print(p)
plot(xdata, sum_gauss(xdata, *p), label="Fitted curve")
for i in range(3):
    plot(xdata, sum_gauss(xdata, *p[3*i:3*i+3]), label="Peak %d" % (i+1))
legend()

# Conclusion

Fitting curves requires the following steps:

* get the data points x_data and y_data
* define the fitting function as y_data = function (x_data, param)
* choose an initial guess for the set of parameters p0
* run the optimizer
* check the result.


# Fitting data with a background

Beyond simple curve fitting, *silx* offers more builtin fitting tools, to handle initial parameter estimation (including peak detection) and background estimation.

We need to import the ``FitManager`` class and the module defining builtin fit theories (``silx.math.fit.fittheories``).

## Loading data

Lets load an image, and sum all samples along the horizontal dimension to get a 1D curve.


In [ ]:
import fabio
img = fabio.open("medipix.edf").data
imshow(np.log(img))

In [ ]:
ydata = img.sum(axis=1)
xdata = numpy.arange(len(img),dtype="float")
plot(xdata, ydata)
axis([0, xdata.max(), 0, ydata.max()])
title("Sum along the horizontal direction")

## Setting up a FitManager

If we want to fit a single gaussian to this data, we clearly need to remove the constant background signal, at about $y=12000$.  

In [ ]:
from silx.math.fit import FitManager, fittheories
fitman = FitManager()
fitman.setdata(x=xdata, y=ydata)
fitman.loadtheories(fittheories)
fitman.settheory('Gaussians')
fitman.setbackground("Constant")

In *silx 0.3*, background theories are hardcoded in ``FitManager``. In the next release, background theories will be handled the same way as fit theories::

    from silx.math.fit import bgtheories
    ...
    fitman.loadbgtheories(bgtheories)


This will enable adding custom background theories, the same way as we can add custom fit theories.


## Estimating initial parameters

Now that the FitManager is set-up, we need to estimate the initial parameters before running the fit. This is done by simply running the ``estimate`` method of fitmanager. This method uses the estimation function defined in fit theories. The results are stored internaly in the FitManager instance as attribute ``fit_results``.

This attribute stores a list of dictionaries, one per parameter to be fitted.

In [ ]:
fitman.estimate()

for param in fitman.fit_results:
    print("Estimated value for param %s: %f" % (param["name"], param["estimation"]))

The estimation function managed to find 1 peak and correctly estimated its gaussian parameters and the background level.

## Final fit

Running the fit is as simple as running the estimation.

In [ ]:
params, uncertainties, infodict = fitman.runfit()

for param in fitman.fit_results:
    print("Fitted value for param %s: %f" % (param["name"], param["fitresult"]))
    
plot(xdata, ydata, label="Original data")
plot(xdata, params[0] + sum_gauss(xdata, *params[1:]), label="Fitted data")

As you can see, parameters can be accessed as the first value returned by the ``runfit`` method, a simple list of raw values, or again as attribute ``fitman.fit_results``.

## More FitManager features

FitManager offers more features, such as:
 - setting constraints for parameters
 - advanced background models, beyond simple constant or linear: strip or snip backgrounds
 - adding custom fit and background theories, with customised estimation function, customized derivatives... 

# Exercise

    x = numpy.arange(10).astype(numpy.float64)
    y = 0.001 * x**3 + 25.1 * x**2 + 1.2 * x - 25
    
1. Fit degree-2 polynomial to this data
2. Fit a degree-3 polynomial to this data
3. Fit a degree-3 polynomial to this data using FitManager.
4. Print the chi-square value and number of iterations for all previous tasks.


+ Tip: For the 3-rd task, to add a model function to FitManager:

      from silx.math.fit import FitTheory
      from silx.math.fit import FitManager

      def poly3(x, a, b, c, d):
          return a * x**3 + b * x**2 + c*x + d
          
      my_poly3_theory = FitTheory(function=poly3, parameters=["a", "b", "c", "d"])
      
      fitman = FitManager()
      fitman.addtheory(name="my poly", theory=my_poly3_theory)
      ...
      

In [37]:
# 1.
import numpy
from silx.math.fit import leastsq

x = numpy.arange(10).astype(numpy.float64)
y = 0.001 * x**3 + 25.1 * x**2 + 1.2 * x - 25


def poly2(x, a, b, c):
    return a * x**2 + b*x + c

p0 = [1., 1., 1.]

p, cov_matrix = leastsq(poly2, x, y, p0)

print("Parameters [a, b, c]: " + str(p))

# 4.
p, cov_matrix, info = leastsq(poly2, x, y, p0, full_output=True)
print(info["reduced_chisq"])
print(info["niter"])


Parameters [a, b, c]: [ 25.1135       1.15390001 -24.97480002]
0.000441257142858
4


In [39]:
# 2.
def poly3(x, a, b, c, d):
    return a * x**3 + b*x**2 + c*x + d

p0 = [1., 1., 1., 1.]

p, cov_matrix = leastsq(poly3, x, y, p0)

print("Parameters [a, b, c, d]: " + str(p))

# 4.
p, cov_matrix, info = leastsq(poly3, x, y, p0, full_output=True)
print(info["reduced_chisq"])
print(info["niter"])


Parameters [a, b, c, d]: [  1.00000007e-03   2.51000000e+01   1.20000001e+00  -2.50000000e+01]
8.96788047245e-17
5


In [40]:
# 3.
from silx.math.fit import FitTheory
from silx.math.fit import FitManager

def poly3(x, a, b, c, d):
    return a * x**3 + b * x**2 + c*x + d

my_poly3_theory = FitTheory(function=poly3, parameters=["a", "b", "c", "d"])

fitman = FitManager()
fitman.addtheory("my poly", my_poly3_theory)
fitman.settheory("my poly")

fitman.setdata(x, y)

# FitManager.estimate() returns an array of initial parameters
p0 = fitman.estimate()
# ... and as we didn't define an estimation function, it will be an array of ones.
print("Estimated parameters: ", p0)

# FitManager.runfit() returns the same data as leastsq(..., full_output=True) 
p, cov, info = fitman.runfit()

print("Parameters [a, b, c, d]: " + str(p))

for param in fitman.fit_results:
    print("parameter %s: %f" % (param["name"], param["fitresult"]))

# 4.
print(fitman.chisq)
print(fitman.)

Estimated parameters:  [ 1.  1.  1.  1.]
Parameters [a, b, c, d]: [  1.00000028e-03   2.51000000e+01   1.20000001e+00  -2.50000000e+01]
parameter a1: 0.001000
parameter b1: 25.100000
parameter c1: 1.200000
parameter d1: -25.000000
4.7203142867e-17
